## 데이터 불러오기

In [72]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [73]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [74]:
!pip install sentence_transformers

In [75]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [76]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

import nltk
from nltk.tokenize import RegexpTokenizer       
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4") 

FHP_article = pd.read_json('healthcare_project_FHP_article.json')
FHP_article = FHP_article.loc[:,'title':]
FHP_article

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,title,date,abstract
0,Effects of scapular treatment on chronic neck ...,2024 Apr 1,Background: Chronic neck pain (CNP) is a commo...
1,Association between smartphone addiction and m...,2024 Apr 1,Background: The purpose of this study was to c...
2,Effects of Telerehabilitation Combining Diaphr...,2024 Mar 11,Background: Forward head posture and rounded s...
3,Effects of Myofascial Release Technique along ...,2024 Mar 4,The purpose of this randomized controlled tria...
4,Sensory Organization and Postural Control Stra...,2024 Mar 26,Objective: This study aimed to compare sensory...
...,...,...,...
1246,Mechanism of production of gait unsteadiness b...,1975 Feb,The hypothesis is put forward that the disturb...
1247,Effect of forward head inclination on visual o...,1972 Nov,
1248,[The mechanism of elongation of the eye-axis. ...,1972 Aug,
1249,Rheumatoid arthritis--atlanto-axial subluxatio...,1966 Mar 5,"The clinical picture, radiological findings an..."


In [77]:
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer

In [78]:
def date_year(raw):
    date_format = raw.split(" ")[0]
    return date_format
def date_month(raw):
    if len(raw.split(" ")) >= 2:
        date_format = raw.split(" ")[1]
    else:
        date_format = None
    return date_format
def date_date(raw):
    if len(raw.split(" ")) >= 3:
        date_format = raw.split(" ")[2]
    else:
        date_format = None
    return date_format
def word_split(raw,word_list):
    word_list.append(raw.split())
    return word_list

In [79]:
FHP_article.loc[:,'date_year']=FHP_article['date'].apply(date_year)
FHP_article.loc[:,'date_month']=FHP_article['date'].apply(date_month)
FHP_article.loc[:,'date_date']=FHP_article['date'].apply(date_date)

### 특수문자, 숫자, '/n' 제거

In [80]:
import re
def word_cleaning(raw):
    raw = raw.replace("\n"," ")
    patterns = '([^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-z|A-Z|\'| ])'
    text_regex = re.sub(pattern = patterns, repl = "",string=raw)
    return text_regex


In [81]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract'].apply(word_cleaning)

In [82]:
FHP_article

,title,date,abstract,date_year,date_month,date_date,abstract_clean
0,Effects of scapular treatment on chronic neck ...,2024 Apr 1,Background: Chronic neck pain (CNP) is a commo...,2024,Apr,1,Background Chronic neck pain CNP is a common p...
1,Association between smartphone addiction and m...,2024 Apr 1,Background: The purpose of this study was to c...,2024,Apr,1,Background The purpose of this study was to cl...
2,Effects of Telerehabilitation Combining Diaphr...,2024 Mar 11,Background: Forward head posture and rounded s...,2024,Mar,11,Background Forward head posture and rounded sh...
3,Effects of Myofascial Release Technique along ...,2024 Mar 4,The purpose of this randomized controlled tria...,2024,Mar,4,The purpose of this randomized controlled tria...
4,Sensory Organization and Postural Control Stra...,2024 Mar 26,Objective: This study aimed to compare sensory...,2024,Mar,26,Objective This study aimed to compare sensory ...
...,...,...,...,...,...,...,...
1246,Mechanism of production of gait unsteadiness b...,1975 Feb,The hypothesis is put forward that the disturb...,1975,Feb,None,The hypothesis is put forward that the disturb...
1247,Effect of forward head inclination on visual o...,1972 Nov,,1972,Nov,None,
1248,[The mechanism of elongation of the eye-axis. ...,1972 Aug,,1972,Aug,None,
1249,Rheumatoid arthritis--atlanto-axial subluxatio...,1966 Mar 5,"The clinical picture, radiological findings an...",1966,Mar,5,The clinical picture radiological findings and...


### 정규화 : 대소문자 통합

In [83]:
def word_lower(raw):
    abstract_list = raw.split()
    modified_list = []
    for abstract_word in abstract_list :
         modified_list.append(abstract_word.lower())
    return " ".join(modified_list)


In [84]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_clean'].apply(word_lower)

In [85]:
FHP_article

,title,date,abstract,date_year,date_month,date_date,abstract_clean
0,Effects of scapular treatment on chronic neck ...,2024 Apr 1,Background: Chronic neck pain (CNP) is a commo...,2024,Apr,1,background chronic neck pain cnp is a common p...
1,Association between smartphone addiction and m...,2024 Apr 1,Background: The purpose of this study was to c...,2024,Apr,1,background the purpose of this study was to cl...
2,Effects of Telerehabilitation Combining Diaphr...,2024 Mar 11,Background: Forward head posture and rounded s...,2024,Mar,11,background forward head posture and rounded sh...
3,Effects of Myofascial Release Technique along ...,2024 Mar 4,The purpose of this randomized controlled tria...,2024,Mar,4,the purpose of this randomized controlled tria...
4,Sensory Organization and Postural Control Stra...,2024 Mar 26,Objective: This study aimed to compare sensory...,2024,Mar,26,objective this study aimed to compare sensory ...
...,...,...,...,...,...,...,...
1246,Mechanism of production of gait unsteadiness b...,1975 Feb,The hypothesis is put forward that the disturb...,1975,Feb,None,the hypothesis is put forward that the disturb...
1247,Effect of forward head inclination on visual o...,1972 Nov,,1972,Nov,None,
1248,[The mechanism of elongation of the eye-axis. ...,1972 Aug,,1972,Aug,None,
1249,Rheumatoid arthritis--atlanto-axial subluxatio...,1966 Mar 5,"The clinical picture, radiological findings an...",1966,Mar,5,the clinical picture radiological findings and...


### 단어 토큰화

In [86]:
def word_tokenizing(raw):
    tokenizer = RegexpTokenizer("[\s]+", gaps=True) 
    tokens = tokenizer.tokenize(raw)
    tags = nltk.pos_tag(tokens)
    nouns_list = [t[0] for t in tags if t[1] in  ['JJ','JJR','MD','NN','NNP','NNS','PRP','VB','VBD','VBG','VBN','VBP','VBZ','WDT']]
    return nouns_list

In [87]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_clean'].apply(word_tokenizing)

In [88]:
FHP_article['abstract_token']

0       [background, chronic, neck, pain, cnp, is, com...
1       [purpose, study, was, clarify, relationship, s...
2       [background, head, posture, rounded, shoulder,...
3       [purpose, randomized, controlled, trial, was, ...
4       [objective, study, aimed, compare, sensory, or...
                              ...                        
1246    [hypothesis, is, put, disturbance, gait, seen,...
1247                                                   []
1248                                                   []
1249    [clinical, picture, radiological, findings, tr...
1250    [oscillation, eyes, patient, head, is, placed,...
Name: abstract_token, Length: 1251, dtype: object

### 표제어 추출

In [91]:
def Lemmatization(raw):
    lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    for word in raw:
        lemmatized_word = lemmatizer.lemmatize(word)
        lemmatized_list.append(lemmatized_word)
    return lemmatized_list

In [92]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_token'].apply(Lemmatization)

In [93]:
FHP_article.loc[:,'abstract_token']

0       [background, chronic, neck, pain, cnp, is, com...
1       [purpose, study, wa, clarify, relationship, sm...
2       [background, head, posture, rounded, shoulder,...
3       [purpose, randomized, controlled, trial, wa, e...
4       [objective, study, aimed, compare, sensory, or...
                              ...                        
1246    [hypothesis, is, put, disturbance, gait, seen,...
1247                                                   []
1248                                                   []
1249    [clinical, picture, radiological, finding, tre...
1250    [oscillation, eye, patient, head, is, placed, ...
Name: abstract_token, Length: 1251, dtype: object

### 표제어 추출

In [ ]:
from sklearn.model_selection import train_test_split
FHP_article_train,FHP_article_test = train_test_split(FHP_article,test_size=0.4,random_state=42)

In [ ]:
FHP_article_train